In [5]:

import mqbench,torch,torchvision,numpy as np,matplotlib.pyplot as plt,torchvision,torchvision.models as models,timm,timm.models as models
%matplotlib inline
from mqbench.prepare_by_platform import prepare_by_platform
from mqbench.prepare_by_platform import BackendType
from mqbench.utils.state import enable_calibration
from mqbench.utils.state import enable_quantization
from mqbench.convert_deploy import convert_deploy
from tqdm import tqdm
from mqbench.fake_quantize.lsq import LearnableFakeQuantize

In [6]:
import torch,torch.nn as nn
class MyModel(nn.Module):
    def forward(self,x,y):
        return torch.matmul(x,y)
        # return x + y
model = MyModel()
model

MyModel()

In [7]:


extra_qconfig_dict = {
    'w_observer': 'MinMaxObserver',
    'a_observer': 'EMAMinMaxObserver',
    'w_fakequantize': 'FixedFakeQuantize',
    'a_fakequantize': 'LearnableFakeQuantize',
    'w_qscheme': {
        'bit': 8,
        'symmetry': True,
        'per_channel': True,
        'pot_scale': False
    },
    'a_qscheme': {
        'bit': 8,
        'symmetry': True,
        'per_channel': False,
        'pot_scale': False
    }
}
device = torch.device('cpu')
prepare_custom_config_dict = {'extra_qconfig_dict': extra_qconfig_dict}
model = prepare_by_platform(model, BackendType.Vitis, prepare_custom_config_dict).to(device)
model

[MQBENCH] INFO: Quantize model Scheme: BackendType.Vitis Mode: Training
[MQBENCH] INFO: Weight Quant Scheme is overrided!
[MQBENCH] INFO: Activation Quant Scheme is overrided!
[MQBENCH] INFO: Weight Qconfig:
    FakeQuantize: FixedFakeQuantize Params: {}
    Oberver:      MinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: True / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Activation Qconfig:
    FakeQuantize: LearnableFakeQuantize Params: {}
    Oberver:      EMAMinMaxObserver Params: Symmetric: True / Bitwidth: 8 / Per channel: False / Pot scale: False / Extra kwargs: {}
[MQBENCH] INFO: Bias Qconfig:
    TqtFakeQuantize with MinMaxObserver
[MQBENCH] INFO: Replace module to qat module.
[MQBENCH] INFO: Add matmul_1 to output quantize
[MQBENCH] INFO: Insert act quant matmul_1_post_act_fake_quantizer


GraphModule(
  (matmul_1_post_act_fake_quantizer): LearnableFakeQuantize(
    fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric, ch_axis=-1, scale=Parameter containing:
    tensor([1.], requires_grad=True), zero_point=Parameter containing:
    tensor([0.], requires_grad=True)
    (activation_post_process): EMAMinMaxObserver(min_val=inf, max_val=-inf ch_axis=-1 pot=False)
  )
)

In [8]:
model

GraphModule(
  (matmul_1_post_act_fake_quantizer): LearnableFakeQuantize(
    fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric, ch_axis=-1, scale=Parameter containing:
    tensor([1.], requires_grad=True), zero_point=Parameter containing:
    tensor([0.], requires_grad=True)
    (activation_post_process): EMAMinMaxObserver(min_val=inf, max_val=-inf ch_axis=-1 pot=False)
  )
)

In [9]:
import torch,torch.nn as nn,torchvision as tv
model = tv.models.resnet18()

def _swap_ff_with_fxff(model: torch.nn.Module) -> None:
    r""" Swap FloatFunctional with FXFloatFunctional
    """
    modules_to_swap = []
    for name, module in model.named_children():
        if isinstance(module, torch.nn.quantized.FloatFunctional):
            modules_to_swap.append(name)
            print(name)
        else:
            _swap_ff_with_fxff(module)

    for name in modules_to_swap:
        del model._modules[name]
        model._modules[name] = torch.nn.quantized.FXFloatFunctional()

_swap_ff_with_fxff(model) # 